# Artificial Neural Network Play

An exploration of artificial neural networks. I already have an understanding on artificial neural networks from my final year at university, but I think that understanding is quite theoretical and shallow. I would like to play with neural networks so that I become familiar with their intricacies and gotchas. The purpose then of this document is to start from the very bottom and work my way up, developing the components of and then a whole artificial neural network from scratch before moving onto using established libraries (PyBrain, scikit-learn, etc.) and employing optimisations, such as using Theano.

There are a couple of tutorials, articles and such which I might be following closely, along with other more rigorous sources. They are:
 - https://datajobs.com/data-science-repo/Neural-Net-%5BCarlos-Gershenson%5D.pdf
 - http://www.wildml.com/2015/09/implementing-a-neural-network-from-scratch/
 - http://andrew.gibiansky.com/blog/machine-learning/coding-intro-to-nns/
 - http://iamtrask.github.io/2015/07/12/basic-python-network/
 
I will endeavour to cite other sources I use as I go.